In [1]:
# revise from https://dp.tdhopper.com/nonparametric-lda/

In [2]:
%matplotlib inline
%precision 2

'%.2f'

# Nonparametric Latent Dirichlet Allocation

_Latent Dirichlet Allocation_is a [generative](https://en.wikipedia.org/wiki/Generative_model) model for topic modeling. Given a collection of documents, an LDA inference algorithm attempts to determined (in an unsupervised manner) the topics discussed in the documents. It makes the assumption that each document is generated by a probability model, and, when doing inference, we try to find the parameters that best fit the model (as well as unseen/latent variables generated by the model). If you are unfamiliar with LDA, Edwin Chen has a [friendly introduction](http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/) you should read.


Because LDA is a _generative_ model, we can simulate the construction of documents by forward-sampling from the model. The generative algorithm is as follows (following [Heinrich](http://www.arbylon.net/publications/text-est.pdf)):

* for each topic $k\in [1,K]$ do
    * sample term distribution for topic $\overrightarrow \phi_k \sim \text{Dir}(\overrightarrow \beta)$
* for each document $m\in [1, M]$ do
    * sample topic distribution for document $\overrightarrow\theta_m\sim \text{Dir}(\overrightarrow\alpha)$
    * sample document length $N_m\sim\text{Pois}(\xi)$
    * for all words $n\in [1, N_m]$ in document $m$ do
        * sample topic index $z_{m,n}\sim\text{Mult}(\overrightarrow\theta_m)$
        * sample term for word $w_{m,n}\sim\text{Mult}(\overrightarrow\phi_{z_{m,n}})$
        
You can implement this with [a little bit of code](https://gist.github.com/tdhopper/521006b60e1311d45509) and start to simulate documents.

In LDA, we assume each word in the document is generated by a two-step process:

1. Sample a topic from the topic distribution for the document.
2. Sample a word from the term distribution from the topic. 

When we fit the LDA model to a given text corpus with an inference algorithm, our primary objective is to find the set of topic distributions $\underline \Theta$, term distributions $\underline \Phi$ that generated the documents, and latent topic indices $z_{m,n}$ for each word.

To run the generative model, we need to specify each of these parameters:

In [3]:
vocabulary=[]
for ii in range(100):
    vocabulary.append('V'+ str(ii))
print(vocabulary)


['V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99']


In [4]:
#vocabulary = ['A', 'B', 'C', "D", "E", "F"]
num_terms = len(vocabulary)
num_topics = 2 # K
num_documents = 5 # M
mean_document_length = 5000 # xi
term_dirichlet_parameter = 1 # beta
topic_dirichlet_parameter = 1 # alpha

The term distribution vector $\underline\Phi$ is a collection of samples from a Dirichlet distribution. This describes how our 3 terms are distributed across each of the two topics.

In [5]:
from scipy.stats import dirichlet, poisson
from numpy import round
from collections import defaultdict
from random import choice as stl_choice

/Users/cwu/opt/anaconda3/envs/HDP/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
term_dirichlet_vector = num_terms * [term_dirichlet_parameter]

In [7]:
term_dirichlet_vector = num_terms * [term_dirichlet_parameter]
term_distributions = dirichlet(term_dirichlet_vector, 10).rvs(size=num_topics)  # for each topic
#print(term_distributions)

Each document corresponds to a categorical distribution across this distribution of topics (in this case, a 2-dimensional categorical distribution). This categorical distribution is a _distribution of distributions_; we could look at it as a Dirichlet process!

The base base distribution of our Dirichlet process is a uniform distribution of topics (remember, topics are term distributions). 

In [8]:
base_distribution = lambda: stl_choice(term_distributions)
# A sample from base_distribution is a distribution over terms
# Each of our two topics has equal probability
from collections import Counter
for topic, count in Counter([tuple(base_distribution()) for _ in range(10000)]).most_common():
    print("count:", count, "topic:", [round(prob, 2) for prob in topic])

count: 5029 topic: [0.01, 0.02, 0.0, 0.01, 0.02, 0.02, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.03, 0.01, 0.0, 0.02, 0.0, 0.02, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.03, 0.03, 0.01, 0.01, 0.0, 0.05, 0.01, 0.02, 0.01, 0.03, 0.01, 0.02, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.02, 0.0, 0.02, 0.01, 0.01, 0.0, 0.01, 0.0, 0.0, 0.0, 0.01, 0.02, 0.01, 0.01, 0.0, 0.02, 0.01, 0.02, 0.02, 0.01, 0.02, 0.01, 0.0, 0.02, 0.0, 0.01, 0.01, 0.01, 0.02, 0.0, 0.0, 0.02, 0.02, 0.01, 0.01, 0.0, 0.01, 0.0, 0.02, 0.0, 0.0, 0.01, 0.0, 0.01, 0.02, 0.01, 0.0]
count: 4971 topic: [0.02, 0.0, 0.01, 0.02, 0.01, 0.0, 0.0, 0.02, 0.0, 0.0, 0.01, 0.03, 0.0, 0.01, 0.02, 0.01, 0.01, 0.0, 0.03, 0.01, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.03, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.01, 0.01, 0.02, 0.0, 0.02, 0.0, 0.02, 0.0, 0.02, 0.0, 0.0, 0.01, 0.0, 0.02, 0.0, 0.01, 0.03, 0.05, 0.01, 0.02, 0.0, 0.01, 0.03, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 

Recall that a sample from a Dirichlet process is a distribution that approximates (but varies from) the base distribution. In this case, a sample from the Dirichlet process will be a distribution over topics that varies from the uniform distribution we provided as a base. If we use the stick-breaking metaphor, we are effectively breaking a stick one time and the size of each portion corresponds to the proportion of a topic in the document.

To construct a sample from the DP, we need to [again define our DP class](/dirichlet-distribution/):

In [9]:
from scipy.stats import beta
from numpy.random import choice

class DirichletProcessSample():
    def __init__(self, base_measure, alpha):
        self.base_measure = base_measure
        self.alpha = alpha
        
        self.cache = []
        self.weights = []
        self.total_stick_used = 0.

    def __call__(self):
        remaining = 1.0 - self.total_stick_used
        i = DirichletProcessSample.roll_die(self.weights + [remaining])
        #print("i")
        #print(i)
        if i is not None and i < len(self.weights) :
            #print("if")
            #print("cache")
            #print(self.cache)
            return self.cache[i]
        else:
            #print("else")
            stick_piece = beta(1, self.alpha).rvs() * remaining
            self.total_stick_used += stick_piece
            self.weights.append(stick_piece)
            new_value = self.base_measure()
            self.cache.append(new_value)
            #print("cache")
            #print(self.cache)
            return new_value
      
    @staticmethod 
    def roll_die(weights):
        if weights:
            #print("weights")
            #print(weights)
            return choice(range(len(weights)), p=weights)
        else:
            #print("None")
            return None

For each document, we will draw a topic distribution from the Dirichlet process:

In [10]:
topic_distribution = DirichletProcessSample(base_measure=base_distribution, 
                                          alpha=topic_dirichlet_parameter)

A sample from this _topic_ distribution is a _distribution over terms_. However, unlike our base distribution which returns each term distribution with equal probability, the topics will be unevenly weighted.

In [11]:
for topic, count in Counter([tuple(topic_distribution()) for _ in range(10000)]).most_common():
    print("count:", count, "topic:", [round(prob, 2) for prob in topic])

count: 6056 topic: [0.02, 0.0, 0.01, 0.02, 0.01, 0.0, 0.0, 0.02, 0.0, 0.0, 0.01, 0.03, 0.0, 0.01, 0.02, 0.01, 0.01, 0.0, 0.03, 0.01, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.03, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.01, 0.01, 0.02, 0.0, 0.02, 0.0, 0.02, 0.0, 0.02, 0.0, 0.0, 0.01, 0.0, 0.02, 0.0, 0.01, 0.03, 0.05, 0.01, 0.02, 0.0, 0.01, 0.03, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.01, 0.0, 0.01, 0.0, 0.0, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0, 0.0, 0.04, 0.0, 0.01, 0.04, 0.01, 0.0, 0.01]
count: 3944 topic: [0.01, 0.02, 0.0, 0.01, 0.02, 0.02, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.03, 0.01, 0.0, 0.02, 0.0, 0.02, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.03, 0.03, 0.01, 0.01, 0.0, 0.05, 0.01, 0.02, 0.01, 0.03, 0.01, 0.02, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.02, 0.0, 0.02, 0.01, 0.01, 0.0, 0.01, 0.0, 0.0, 0.0, 0.01, 0.02, 0.01, 0.01, 0.0, 0.02, 0.01, 0.0

In [ ]:
# make the topic distribution uneven

To generate each word in the document, we draw a sample topic from the topic distribution, and then a term from the term distribution (topic). 

In [12]:
topic_index = defaultdict(list)
documents = defaultdict(list)

for doc in range(num_documents):
    topic_distribution_rvs = DirichletProcessSample(base_measure=base_distribution, 
                                                    alpha=topic_dirichlet_parameter)
    document_length = poisson(mean_document_length).rvs()
    for word in range(document_length):
        topic_distribution = topic_distribution_rvs()
        topic_index[doc].append(tuple(topic_distribution))
        documents[doc].append(choice(vocabulary, p=topic_distribution))

Here are the documents we generated:

In [173]:
import pandas as pd
## data to dataframe
df_all=pd.DataFrame(vocabulary, columns=["word"])
i=0
print(df_all)
for doc in documents.values():
    i+=1
    #counter=Counter(doc)
    #scounter=sorted(counter.items())
    #print(scounter)
    scount=[]
    for vv in vocabulary:
        scount.append(doc.count(vv))
    #df=pd.DataFrame(scounter, columns=['word', "freq"])
    df=pd.DataFrame(list(zip(vocabulary, scount)), columns=['word', "freq"])
    if(all(df['word']==df_all['word'])):
        df_all['doc'+str(i)]=df['freq']
    else:
        print("not match")
    
    #print((sorted(counter.items())[0][1]))
    
df_all2=df_all.iloc[:,range(1,num_documents+1)].T
print(df_all2)
df_all2.to_csv(f"~/test_data_topic_model_{num_terms}voc_{num_topics}topics_{num_documents}docs_{mean_document_length}words_beta{term_dirichlet_parameter}_alpha{topic_dirichlet_parameter}.csv"
              ,header=False)
               
               

   word
0    V0
1    V1
2    V2
3    V3
4    V4
..  ...
95  V95
96  V96
97  V97
98  V98
99  V99

[100 rows x 1 columns]
      0    1   2   3    4   5   6   7   8   9   ...  90  91  92   93  94  95  \
doc1  63   88  23  47   83  54  46  17  19  52  ...  13  80   4   56  47  28   
doc2  66    4  62  74   44  19  14  86  13   3  ...  36  16   3  194  21  35   
doc3  55  102  11  46  104  80  54   1  27  55  ...   5  73   3   30  54  24   
doc4  55   74  26  58   62  46  31  30  25  30  ...  11  47   5   83  48  32   
doc5  62   52  33  61   66  42  45  41  17  27  ...  20  37   5  114  26  43   

       96  97  98  99  
doc1   86  91  14  12  
doc2  212  35  20  50  
doc3   83  79  25  10  
doc4  135  93  33  26  
doc5  157  73  22  24  

[5 rows x 100 columns]


We can see how each topic (term-distribution) is distributed across the documents:

In [14]:
for i, doc in enumerate(Counter(term_dist).most_common() for term_dist in topic_index.values()):
    print("Doc:", i)
    for topic, count in doc:
        print(5*" ", "count:", count, "topic:", [round(prob, 2) for prob in topic])

Doc: 0
      count: 4909 topic: [0.02, 0.0, 0.01, 0.02, 0.01, 0.0, 0.0, 0.02, 0.0, 0.0, 0.01, 0.03, 0.0, 0.01, 0.02, 0.01, 0.01, 0.0, 0.03, 0.01, 0.01, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.03, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.01, 0.01, 0.02, 0.0, 0.02, 0.0, 0.02, 0.0, 0.02, 0.0, 0.0, 0.01, 0.0, 0.02, 0.0, 0.01, 0.03, 0.05, 0.01, 0.02, 0.0, 0.01, 0.03, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.01, 0.0, 0.01, 0.0, 0.0, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0, 0.0, 0.04, 0.0, 0.01, 0.04, 0.01, 0.0, 0.01]
      count: 81 topic: [0.01, 0.02, 0.0, 0.01, 0.02, 0.02, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.03, 0.01, 0.0, 0.02, 0.0, 0.02, 0.01, 0.01, 0.0, 0.0, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.03, 0.03, 0.01, 0.01, 0.0, 0.05, 0.01, 0.02, 0.01, 0.03, 0.01, 0.02, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.02, 0.0, 0.02, 0.01, 0.01, 0.0, 0.01, 0.0, 0.0, 0.0, 0.01, 0.02, 0.01, 0.01, 0.0

In [175]:
tmp={}
for i, doc in enumerate(sorted(Counter(term_dist).items()) for term_dist in topic_index.values()):
    print("Doc:", i)
    for jj in topics_all:
        doc2=dict(doc)
        if tuple(jj) in doc2.keys():
            count=doc2[tuple(jj)]
            tt=[round(prob, 3) for prob in jj]
            if i==0:
                tmp[str(tt)]=[count]  
            else:
                cc=tmp[str(tt)]
                cc.append(count)
                tmp[str(tt)]=cc
        else:
            tt=[round(prob, 3) for prob in jj]
            if i==0:
                tmp[str(tt)]=[0]  
            else:
                cc=tmp[str(tt)]
                cc.append(0)
                tmp[str(tt)]=cc
            
prop_table=pd.DataFrame(tmp)
print(prop_table)      
prop_table.to_csv(f"~/test_prop_table_topic_model_{num_terms}voc_{num_topics}topics_{num_documents}docs_{mean_document_length}words_beta{term_dirichlet_parameter}_alpha{topic_dirichlet_parameter}.csv")

Doc: 0
Doc: 1
Doc: 2
Doc: 3
Doc: 4
   [0.016, 0.0, 0.011, 0.015, 0.008, 0.003, 0.002, 0.016, 0.002, 0.001, 0.013, 0.034, 0.0, 0.008, 0.018, 0.01, 0.014, 0.004, 0.027, 0.014, 0.009, 0.002, 0.005, 0.012, 0.006, 0.006, 0.011, 0.008, 0.011, 0.01, 0.018, 0.008, 0.026, 0.004, 0.001, 0.004, 0.001, 0.019, 0.001, 0.011, 0.009, 0.019, 0.002, 0.021, 0.005, 0.015, 0.004, 0.024, 0.004, 0.002, 0.005, 0.001, 0.019, 0.002, 0.005, 0.032, 0.048, 0.007, 0.019, 0.003, 0.01, 0.025, 0.008, 0.01, 0.0, 0.005, 0.001, 0.004, 0.004, 0.016, 0.0, 0.006, 0.004, 0.011, 0.005, 0.0, 0.023, 0.016, 0.023, 0.006, 0.01, 0.008, 0.01, 0.003, 0.004, 0.0, 0.004, 0.003, 0.009, 0.009, 0.007, 0.004, 0.001, 0.042, 0.005, 0.007, 0.041, 0.006, 0.004, 0.008]  \
0                                                938                                                                                                                                                                                                                                

In [130]:
prop_table

,"[0.011, 0.002, 0.037, 0.004, 0.01, 0.062, 0.095, 0.013, 0.038, 0.006, 0.02, 0.05, 0.017, 0.019, 0.001, 0.01, 0.002, 0.008, 0.009, 0.032, 0.001, 0.012, 0.008, 0.022, 0.009, 0.001, 0.046, 0.031, 0.045, 0.012, 0.02, 0.016, 0.02, 0.007, 0.008, 0.001, 0.008, 0.006, 0.018, 0.018, 0.014, 0.007, 0.001, 0.083, 0.009, 0.015, 0.082, 0.013, 0.008, 0.016]","[0.02, 0.046, 0.002, 0.015, 0.036, 0.03, 0.021, 0.001, 0.01, 0.02, 0.008, 0.017, 0.027, 0.004, 0.001, 0.01, 0.003, 0.005, 0.051, 0.011, 0.006, 0.033, 0.003, 0.032, 0.015, 0.022, 0.004, 0.005, 0.025, 0.001, 0.007, 0.019, 0.006, 0.011, 0.061, 0.062, 0.02, 0.018, 0.0, 0.09, 0.02, 0.037, 0.02, 0.05, 0.021, 0.029, 0.004, 0.013, 0.028, 0.003]","[0.033, 0.0, 0.022, 0.031, 0.015, 0.006, 0.005, 0.032, 0.004, 0.002, 0.026, 0.068, 0.0, 0.016, 0.037, 0.021, 0.028, 0.008, 0.055, 0.028, 0.017, 0.003, 0.01, 0.025, 0.013, 0.013, 0.021, 0.016, 0.023, 0.02, 0.036, 0.016, 0.053, 0.009, 0.002, 0.008, 0.003, 0.039, 0.001, 0.022, 0.018, 0.038, 0.005, 0.043, 0.01, 0.031, 0.008, 0.048, 0.009, 0.004]"
0,236,739,35
1,838,0,123
2,212,818,16
3,410,0,551
4,26,13,996


To recap: for each document we draw a _sample_ from a Dirichlet _Process_. The base distribution for the Dirichlet process is a categorical distribution over term distributions; we can think of the base distribution as an $n$-sided die where $n$ is the number of topics and each side of the die is a distribution over terms for that topic. By sampling from the Dirichlet process, we are effectively reweighting the sides of the die (changing the distribution of the topics).

For each word in the document, we draw a _sample_ (a term distribution) from the distribution (over term distributions) _sampled_ from the Dirichlet process (with a distribution over term distributions as its base measure). Each term distribution uniquely identifies the topic for the word. We can sample from this term distribution to get the word.

Given this formulation, we might ask if we can roll an _infinite_ sided die to draw from an unbounded number of topics (term distributions). We can do exactly this with a _Hierarchical_ Dirichlet process. Instead of the base distribution of our Dirichlet process being a _finite_ distribution over topics (term distributions) we will instead make it an infinite Distribution over topics (term distributions) by using yet another Dirichlet process! This base Dirichlet process will have as its base distribution a Dirichlet _distribution_ over terms. 

We will again draw a _sample_ from a Dirichlet _Process_ for each document. The base distribution for the Dirichlet process is itself a Dirichlet process whose base distribution is a Dirichlet distribution over terms. (Try saying that 5-times fast.) We can think of this as a countably infinite die each side of the die is a distribution over terms for that topic. The sample we draw is a topic (distribution over terms).

For each word in the document, we will draw a _sample_ (a term distribution) from the distribution (over term distributions) _sampled_ from the Dirichlet process (with a distribution over term distributions as its base measure). Each term distribution uniquely identifies the topic for the word. We can sample from this term distribution to get the word.

These last few paragraphs are confusing! Let's illustrate with code.

Our documents were generated by an unspecified number of topics, and yet the topics were shared across the 5 documents. This is the power of the hierarchical Dirichlet process!

This non-parametric formulation of Latent Dirichlet Allocation was first published by [Yee Whye Teh et al](http://www.cs.berkeley.edu/~jordan/papers/hdp.pdf). 

Unfortunately, forward sampling is the easy part. Fitting the model on data requires [complex MCMC](http://psiexp.ss.uci.edu/research/papers/sciencetopics.pdf) or [variational inference](http://www.cs.princeton.edu/~chongw/papers/WangPaisleyBlei2011.pdf). There are a [limited](http://www.stats.ox.ac.uk/~teh/software.html) [number](https://github.com/shuyo/iir/blob/master/lda/hdplda2.py) of [implementations](https://github.com/renaud/hdp-faster) [of HDP-LDA](http://www.arbylon.net/resources.html) available, and none of them are great. 